# NN神经网络
- 神经网络可以使用torch.nn包来构建
### 一个典型的神经网络的训练过程:
- 定义具有一些学习参数或权重的神经网络
- 迭代输入数据集
- 通过网络处理输入
- 计算损失函数
- 反向传播
- 更新网络的参数,权重

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 构建神经网络
class Net(nn.Module):  # 只要继承了Module这个类,子类定义了forward()函数,backward()函数就会被自动实现
    def __init__(self):
        super(Net,self).__init__()  # 貌似是固定的代码写法
        # 核心
        # 构建卷积层
        self.conv1=nn.Conv2d(1,6,5)  # 卷积层的参数: 第一个为输入图片的通道数,第二个为输出的通道数,第三个表示卷积核的大小(5*5)
        self.conv2=nn.Conv2d(6,16,5)
        # 全连接层
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    
    def forward(self,x):
        # 对于输入的应用2D最大池化层
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))  # (2,2)是池化操作的步幅
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:]  # 获取除去数量维度外的所有维度
        num_features=1
        for s in size:
            num_features*=s
        return num_features

In [3]:
net=Net()
net

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)

In [4]:
params=list(net.parameters())  # net.parameters()返回了网络的可学习参数
print(len(params))
print(params[0].size())  # 第一个卷积层的权重

10
torch.Size([6, 1, 5, 5])


In [5]:
# 调整输入的数据集大小
input=Variable(torch.randn(1,1,32,32))
out=net(input)
out

Variable containing:
-0.1619 -0.0549  0.0681  0.0388 -0.1209 -0.1328 -0.0782 -0.0918  0.0185  0.0955
[torch.FloatTensor of size 1x10]

In [6]:
net.zero_grad()
out.backward(torch.randn(1,10))

## 损失函数

In [7]:
output=net(input)
target=Variable(torch.arange(1,11))
criterion=nn.MSELoss()
loss=criterion(output,target)
loss

Variable containing:
 38.8038
[torch.FloatTensor of size 1]

In [8]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

## 反向传播

In [9]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
 -1.5622
 -7.3783
  4.8139
 -9.3312
  2.3244
  3.8294
[torch.FloatTensor of size 6]



## 更新权重

In [10]:
import torch.optim as optim
#  新建一个优化器,指定要调整的参数和学习率
optimizer=optim.SGD(net.parameters(),lr=0.01)
# 训练过程:
optimizer.zero_grad()  # 首先梯度清零与net.zero_grad()效果一样
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step()